# FBS unexpected behavior on 2024-11-19

Expected behavior: 
using the [fbs_config_minimal_fieldsurvey.py](https://github.com/lsst-ts/ts_config_ocs/blob/5b7f4ebd83b0550d6da6aa180d11c955163991f4/Scheduler/feature_scheduler/maintel/fbs_config_minimal_fieldsurvey.py) configuration, 
the observers would slew to the field to be observed, and take sequences of 5-r, 5-i, 10-y band visits.
The field choice is primarily driven by the slew basis functions.
The first field, ECDFS, was "chosen" by slewing to the field and starting the scheduler. 

This went as expected at first - ECDFS was chosen and as sequence acquired. This was consistent with behavior on previous nights. 
However after the first sequence of visits, the FBS chosen a totally different field as a target and started slewing the telescope quite far away. This was unexpected.


In [1]:
import os
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.summit.utils import ConsDbClient

import sys
sys.path.insert(0, '/home/l/lynnej/u/repos/rubin_scheduler')

import pickle
from urllib.parse import urlparse
from lsst.resources import ResourcePath
from lsst_efd_client import EfdClient
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site
from rubin_scheduler.scheduler.example import get_ideal_model_observatory

from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions

os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'

days_to_seconds = 24*60*60

In [3]:
# We need to find a range of times to query. 
# If it's early in the night, this can be the easiest - to just use the dayobs. 

In [12]:
# Set a range of times values to query efd and consdb .. 
day_obs = "2024-11-19"

t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = Time(f"{day_obs}T12:00:00", format='isot', scale='utc') + TimeDelta(18 * 60 * 60, format='sec')

print(f"Querying from {t_start.iso} to {t_end.iso}")

Querying from 2024-11-19 12:00:00.000 to 2024-11-20 06:00:00.000


In [31]:
# Get targets from the EFD 

efd = 'usdf_efd'

efd_client = EfdClient(efd)

salindex = 1
topic = 'lsst.sal.Scheduler.logevent_target'
fields = await efd_client.get_fields(topic)
targets = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
delta_target = (targets.index.values[1:] - targets.index.values[:-1]) / np.timedelta64(1, 's')
targets['delta_target'] = np.concatenate([np.array([0]), delta_target])
#print(fields)
targets['count'] = np.arange(0, len(targets))
tcols = ['delta_target', 'requestMjd', 'ra', 'decl', 'filter', 'exposureTimes0', 'slewTime', 'skyAngle', 'airmass', 'moonRa', 'skyBrightness', 'seeing', 'note', 'targetName', 'targetId', 'count']
targets[tcols]

,delta_target,requestMjd,ra,decl,filter,exposureTimes0,slewTime,skyAngle,airmass,moonRa,skyBrightness,seeing,note,targetName,targetId,count
2024-11-20 01:15:55.009141+00:00,0.000000,0,53.180494,-28.065668,r_03,30,13.645975,220.697594,1.294080,0,21.170198,0,,,0,0
2024-11-20 01:15:56.187442+00:00,1.178301,0,53.336524,-28.102853,r_03,30,13.645976,220.709895,1.294080,0,21.170198,0,,,0,1
2024-11-20 01:17:04.439697+00:00,68.252255,0,53.147960,-28.099419,r_03,30,13.645975,220.744093,1.294080,0,21.170198,0,,,0,2
2024-11-20 01:17:56.828369+00:00,52.388672,0,53.160909,-28.148259,r_03,30,13.645976,220.799297,1.294080,0,21.170198,0,,,0,3
2024-11-20 01:18:48.097800+00:00,51.269431,0,52.934119,-28.164413,r_03,30,13.645976,220.864776,1.294080,0,21.170198,0,,,0,4
2024-11-20 01:19:38.385989+00:00,50.288189,0,53.204015,-28.119144,i_06,30,120.000000,220.073638,1.294080,0,20.238480,0,,,0,5
2024-11-20 01:20:29.528241+00:00,51.142252,0,53.018985,-27.948421,i_06,30,120.000000,219.908417,1.294080,0,20.238480,0,,,0,6
2024-11-20 01:22:20.733268+00:00,111.205027,0,53.038973,-28.009154,i_06,30,120.000000,219.976527,1.294080,0,20.238480,0,,,0,7
2024-11-20 01:23:12.843585+00:00,52.110317,0,53.232573,-28.248498,i_06,30,120.000000,220.220765,1.294080,0,20.238480,0,,,0,8
2024-11-20 01:24:03.952346+00:00,51.108761,0,53.112555,-28.143517,i_06,30,120.000000,220.121123,1.294080,0,20.238480,0,,,0,9


In [32]:
# The targets -- up until the target at RA=300 / Dec = -40 -- were all acquired as visits. 

os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
os.environ["no_proxy"] += ",.consdb"

consdb = ConsDbClient()
instrument = 'lsstcomcam'
day_obs_int = day_obs.replace('-', '')
visit_query = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''
visits = consdb.query(visit_query).to_pandas()

vv = pd.DataFrame(visits.query('science_program == "BLOCK-320"'))
delta_visit = vv.exp_midpt_mjd.values[1:] - vv.exp_midpt_mjd.values[:-1]
delta_visit = np.concatenate([np.array([0]), delta_visit])
vv['delta_visit'] = delta_visit * 24 * 60 * 60
print(vv.exp_midpt_mjd.min(), vv.exp_midpt_mjd.max(), (vv.exp_midpt_mjd.max() - vv.exp_midpt_mjd.min()) * 24 * 60)
vcols = ['exposure_name', 'delta_visit', 'exp_midpt_mjd', 'band', 's_ra', 's_dec', 'sky_rotation', 'target_name']
# trim down visits (since we know where the last relevant one was
vv = vv.query('exp_midpt_mjd <= 60634.308665')
display(HTML(vv[vcols].to_html()))

60634.053732694636 60634.337199112764 408.19164210464805


,exposure_name,delta_visit,exp_midpt_mjd,band,s_ra,s_dec,sky_rotation,target_name
80,CC_O_20241119_000081,0.000000,60634.053733,r,53.180494,-28.065668,220.701571,ECDFS
81,CC_O_20241119_000082,51.606841,60634.054330,r,53.336524,-28.102853,220.713876,ECDFS
82,CC_O_20241119_000083,51.305382,60634.054924,r,53.147960,-28.099419,220.747979,ECDFS
83,CC_O_20241119_000084,50.602505,60634.055509,r,53.160909,-28.148259,220.803113,ECDFS
84,CC_O_20241119_000085,51.301789,60634.056103,r,52.934119,-28.164413,220.868498,ECDFS
85,CC_O_20241119_000086,111.028605,60634.057388,i,53.204015,-28.119144,220.077326,ECDFS
86,CC_O_20241119_000087,52.244603,60634.057993,i,53.018985,-27.948421,219.912083,ECDFS
87,CC_O_20241119_000088,51.000017,60634.058583,i,53.038973,-28.009154,219.980186,ECDFS
88,CC_O_20241119_000089,51.692838,60634.059182,i,53.232573,-28.248498,220.224401,ECDFS
89,CC_O_20241119_000090,52.403943,60634.059788,i,53.112555,-28.143517,220.124678,ECDFS


In [20]:
# The visits track the targets up to the point where the FBS tried to point at the Rubin_SV_300_-4 field at RA=300, Dec=-40
# What was the alt/az of this pointing at that time?  
# (there was some concern that the FBS was trying to point at something that had not yet risen ..)

from rubin_scheduler.utils import approx_ra_dec2_alt_az
from rubin_scheduler.utils import Site

lsst = Site('LSST')
mjd = 60634.066606
ra= 300.064850	
dec = -40.965661

alt, az = approx_ra_dec2_alt_az(ra, dec, lsst.latitude, lsst.longitude, mjd)
print(alt, az)

# So Rubin_SV_300 target was at altitude of 31 degrees -- so it was really low, but not forbidden by the altitude limits available (20 deg).

31.591484403321804 237.83485088146006


In [24]:
# What was the scheduler snapshot looking like? We only have one from the start of the first target
# Snapshots are saved when the FBS is queried for "request_observation" 
# This currently happens for two calls in a row, then the resulting sequences are parsed in sets of twos by the Scheduler queue

# Get the snapshots 

topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]

snapshots = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
print(len(snapshots))
display(HTML(snapshots.to_html()))


1


,url
2024-11-20 01:15:53.302238+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/19/Scheduler:1_Scheduler:1_2024-11-20T01:16:30.067.p


In [25]:
index = 0 

snapshot_time = snapshots.iloc[index].name
snapshot_mjd = Time(snapshot_time).mjd
url = snapshots.iloc[index].url
print(snapshot_time, url)

# EFD records the summit LFA -- if not at the summit, swap.
bucket = "s3://rubin:"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

# multi-tenant names have colons 
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

result = uri.read()

#unpickle
sched, conditions = pickle.loads(result)

# Just check that these are the right kind of things 
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)

2024-11-20 01:15:53.302238+00:00 https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/19/Scheduler:1_Scheduler:1_2024-11-20T01:16:30.067.p


In [26]:
sched.survey_lists

[[<FieldSurvey survey_name='Rubin_SV_095_-25', RA=1.6580627893946132, dec=-0.4363323129985824 at 0x7fbd952225d0>,
  <FieldSurvey survey_name='Rubin_SV_125_-15', RA=2.181661564992912, dec=-0.2617993877991494 at 0x7fbd9523db10>,
  <FieldSurvey survey_name='DESI_SV3_R1', RA=3.134611336581816, dec=0.0 at 0x7fbd9523e850>,
  <FieldSurvey survey_name='Rubin_SV_225_-40', RA=3.9269908169872414, dec=-0.6981317007977318 at 0x7fbd9523f5d0>,
  <FieldSurvey survey_name='DEEP_A0', RA=3.7699111843077517, dec=-0.2181661564992912 at 0x7fbd95244350>,
  <FieldSurvey survey_name='Rubin_SV_250_2', RA=4.363323129985824, dec=0.03490658503988659 at 0x7fbd95245190>,
  <FieldSurvey survey_name='Baades_Window', RA=4.72795150536607, dec=-0.5241902502285072 at 0x7fbd95245f10>,
  <FieldSurvey survey_name='Rubin_SV_300_-41', RA=5.235987755982989, dec=-0.7155849933176751 at 0x7fbd95246c90>,
  <FieldSurvey survey_name='Rubin_SV_280_-48', RA=4.886921905584122, dec=-0.8377580409572782 at 0x7fbd95254690>,
  <FieldSurvey s

In [28]:
# First visit time matches conditions.mjd, as expected.
print('conditions time', conditions.mjd) 
# Check on alt/az limits in the conditions (to see if anything masked)
print('tel limits', np.degrees(conditions.tel_alt_limits), np.degrees(conditions.tel_az_limits))
# And check on current position of the telescope -- expect this to be ECDFS location, as they pointed there
print('tel pos', np.degrees(conditions.tel_ra), np.degrees(conditions.tel_dec))
print(conditions.current_filter, conditions.mounted_filters)

conditions time 60634.053125
tel limits [20. 84.] [-200.  200.]
tel pos 53.522021594241075 -28.032611187843074
r_03 ['i_06', 'r_03', 'y_04']


In [29]:
# just check on surveys and how many observations they know about .. 
# (there's only one tier in this scheduler configuration)
# Might as well check total # of visits against consdb previous visits 
# (since this is start of the day, these should be all from previous nights)
visit_query = f'''SELECT * FROM cdb_{instrument}.visit1 where day_obs < {day_obs_int} and science_program like '%%BLOCK-320%%' '''
visits = consdb.query(visit_query).to_pandas()

for s in sched.survey_lists[0]:
    # ok, the list is long .. only do this for the ones with non-zero obs for the survey .. 
    if s.extra_features['ObsRecorded_note'].feature > 0:
        print("survey", s.survey_name, np.degrees(s.ra), np.degrees(s.dec))
        # hmm .. not all previous visits are counted .. but perhaps some of the previous visits were bad?
        print("nvisits", s.extra_features['ObsRecorded'].feature, s.extra_features['ObsRecorded_note'].feature, len(visits.query('target_name == @s.survey_name')))

survey ECDFS 53.125 -28.1
nvisits 140 33 72
survey EDFS_comcam 59.10040000000001 -48.73
nvisits 140 63 65


In [11]:
# What times are we talking about .. 
visitidx = 80
targetidx = 0
print(conditions.mjd, vv.exp_midpt_mjd[visitidx], Time(targets.index[targetidx]).utc.mjd)
print("first visit - conditions time")
print((vv.exp_midpt_mjd[visitidx] - conditions.mjd) * days_to_seconds )
print("conditions - target recorded time")
print((conditions.mjd - Time(targets.index[targetidx]).utc.mjd) * days_to_seconds)

60634.053125 60634.053732694636 60634.052720013206
first visit - conditions time
52.504816674627364
conditions - target recorded time
34.990858915261924


In [33]:
# All right - let's go update the scheduler to the current conditions, and ask for an observation. 
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
sched.update_conditions(conditions)
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
# ... so maybe at the start of the night, the conditions pickled in the scheduler match the current conditions. 

# Also, given the times above .. the conditions.mjd is not quite the time the visit is acquired, 
#but it's also not the time the target is recorded .. so let's assume it's the goal visit start time.

60634.053125 60634.053125 0.9341354991430113
60634.053125 60634.053125 0.9341354991430113


In [34]:
obs = sched.request_observation(conditions.mjd, whole_queue=True)

/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:292: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))


In [35]:
# The requested observations match the ECDFS field, and contain the entire sequence. 
print(len(obs))
obs

20


[array([(0, 0.92817472, -0.48983831, 0., 60640.79395833, 30., 'r_03', 3.85189966, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., '', 'ECDFS', 'ECDFS', 0, 0., 0.60794847, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0, 'science', 'BLOCK-320')],
       dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('note', '<U40'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('block_id', '<i8'), ('lmst', '<f8'), ('rotTelPos', '<f8'), ('rotTelPos_backup', '<f8'), ('moonAz', 

In [36]:
# Now -- to request another sequence, for the target at 2024-11-20 01:34:45.081598+00:00
# ... I *think* but am not sure, that this is done before any update is done in terms of observations??
targets.loc["2024-11-20 01:34:45.081598+00:00":"2024-11-20 01:34:45.081598+00:00"][tcols]

,delta_target,requestMjd,ra,decl,filter,exposureTimes0,slewTime,skyAngle,airmass,moonRa,skyBrightness,seeing,note,targetName,targetId,count
2024-11-20 01:34:45.081598+00:00,52.26399,0,300.06485,-40.965661,y_04,30,92.274991,86.289033,1.880831,0,18.037142,0,,,0,20


In [37]:
# .. I don't know what the second call to the FBS would look like. 
# Let's assume we set conditions tel_ra and tel_dec to the location of the last requested observation
conditions.tel_ra = obs[-1]['RA']
conditions.tel_dec = obs[-1]['dec']
conditions.current_filter = obs[-1]['filter']
print(conditions.tel_ra, conditions.tel_dec, conditions.mjd)
# the last observation doesn't have a time ... 
# the target times are tied to queue operations, and this time wouldn't be available when the next target is requested
# what happens if we just use the conditions again at the same time? 

sched.update_conditions(conditions)
obs2 = sched.request_observation(conditions.mjd, whole_queue=True)
obs2

[0.92634766] [-0.49038697] 60634.053125


/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:292: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))


[array([(0, 5.23711961, -0.71498566, 0., 60640.79395833, 30., 'y_04', 1.1685156, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., '', 'Rubin_SV_300_-41', 'Rubin_SV_300_-41', 0, 0., 0.60794847, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0, 'science', 'BLOCK-320')],
       dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('note', '<U40'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('block_id', '<i8'), ('lmst', '<f8'), ('rotTelPos', '<f8'), ('rotTelPos_backup'

In [40]:
# ah .. once we pull the observations out of the queue, 
sched.update_conditions(conditions)
sched._fill_queue()
winner = sched.survey_index
ss = sched.survey_lists[winner[0]][winner[1]]
ss.survey_name, winner
# Okay, we've reproduced the "runaway" behavior -- let's see why this survey won

/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:292: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))


('Rubin_SV_300_-41', [0, 7])

In [41]:
# Look at all of the rewards 
reward_df = sched.make_reward_df(conditions=conditions, accum=True)
summary_df = reward_df.reset_index()
summary_df[['survey_index', 'survey_label', 'survey_reward']].groupby('survey_index').max()

/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:559: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:559: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:559: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:559: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:559: RuntimeWarnin

,survey_label,survey_reward
survey_index,,
0,Rubin_SV_095_-25,NaN
1,Rubin_SV_125_-15,NaN
2,DESI_SV3_R1,NaN
3,Rubin_SV_225_-40,NaN
4,DEEP_A0,NaN
5,Rubin_SV_250_2,NaN
6,Baades_Window,NaN
7,Rubin_SV_300_-41,0.0
8,Rubin_SV_280_-48,0.0


In [42]:
# It's odd that they're all 0 of nan ..
# Let's look at the reward basis value functions for the winner
reward_df.loc[winner[0], winner[1]]

basis_function          basis_function_class  \
list_index survey_index                                                  
0          7                 NotTwilight      NotTwilightBasisFunction   
           7             AltAzShadowMask  AltAzShadowMaskBasisFunction   
           7               Slewtime u_02         SlewtimeBasisFunction   
           7               Slewtime g_01         SlewtimeBasisFunction   
           7               Slewtime r_03         SlewtimeBasisFunction   
           7               Slewtime i_06         SlewtimeBasisFunction   
           7               Slewtime z_03         SlewtimeBasisFunction   
           7                FilterLoaded     FilterLoadedBasisFunction   

                         feasible  max_basis_reward  basis_area  basis_weight  \
list_index survey_index                                                         
0          7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   
           7                 True               0.0    3.357175         0.125   

                         max_accum_reward  accum_area tier_label  \
list_index survey_index                                            
0          7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   
           7                          0.0    3.357175     tier 0   

                             survey_label survey_class  survey_reward  
list_index survey_index                                                
0          7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0  
           7             Rubin_SV_300_-41  FieldSurvey            0.0

And this is where I noticed --- there is no slewtime basis function for y band.
The sequence ends in y band, but then there is no constraint for the slewtime, so all surveys are equally likely. Thus the "runaway" to whichever is first in the list within the tier. 